## Recurrent Neural Network

### $ h_t=f_W\left( h_{t-1},x_t \right) $
* $h_t$: new state
* $f_W$: some function with parameters W
* $h_{t-1}$: old state
* $x_t$: input vector at some time step

### $ h_t=tanh(W_{hh} h_{t-1}+W_{xh} x_t) $
### $ y_t=W_{hy}h_t$
Given list of word vectors: 
#### $ x_1,\cdots,x_{t-1},x_t,x_{t+1},\cdots, x_T $ <br>
At a single time step:
#### $ h_t = \sigma(W^{(hh)}h_{t-1}+W^{(hx)}x_t)$
#### $ \hat{y}_t = softmax(W^{(S)}h_t)$
#### $ \hat{P}(x_{t+1}=v_j  \big|  x_t,\cdots,x_1) = \hat{y}_{t,j}$

### RNN applications
* Language Modeling
* Speech Recognition
* Machine Translation
* Conversation Modeling / Question Answering
* Image/Video Caption
* Image/Music/Dance Generation

### RNN model
* **one to one** <br>
o<br>
$\uparrow$<br>
o<br>
$\uparrow$<br>
o<br>
_**Vanilla Neural Networks**_


* **one to many** <br>
o&emsp; o&emsp;o<br>
$\uparrow$&emsp;$\uparrow$&emsp;$\uparrow$<br>
o$\rightarrow$o$\rightarrow$o<br>
$\uparrow$<br>
o<br>
_**Image Caption(image -> sequence of words)**_


* **many to one** <br>
&emsp;&emsp;&emsp; o<br>
&emsp;&emsp;&emsp; $\uparrow$<br>
o$\rightarrow$o$\rightarrow$o<br>
$\uparrow$&emsp;$\uparrow$&emsp;$\uparrow$<br>
o&emsp; o&emsp;o<br>
_**Sentiment Classification(sequence of word -> sentiment)**_


* **many to many** <br>
&emsp;&emsp;&emsp; o&emsp; o&emsp;o<br>
&emsp;&emsp;&emsp; $\uparrow$&emsp;$\uparrow$&emsp;$\uparrow$<br>
o$\rightarrow$o$\rightarrow$o$\rightarrow$o$\rightarrow$o<br>
$\uparrow$&emsp;$\uparrow$&emsp;$\uparrow$<br>
o&emsp; o&emsp;o<br>
_**Machine Translation(seq of words -> seq of words)**_


* **many to many** <br>
o&emsp; o&emsp;o<br>
$\uparrow$&emsp;$\uparrow$&emsp;$\uparrow$<br>
o$\rightarrow$o$\rightarrow$o<br>
$\uparrow$&emsp;$\uparrow$&emsp;$\uparrow$<br>
o&emsp; o&emsp;o<br>
_**Viedeo Classification on frame level**_<br>